# Проект для «Викишоп»

## Описание исследования

Интернет-магазин «Викишоп» запускает новый сервис. Пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. Клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

## Цель исследования

- Обучить модель классифицировать комментарии как позитивные или негативные. Построить модель со значением метрики качества F1 не меньше 0.75.

## Ход исследования

1. Загрузка и подготовка данных
2. Обучение моделей
2. Выводы

## Описание данных

- 'text' - текст комментарий
- 'toxic' - целевой признак, является ли комментарий токсичным

<a id='section_id'></a>
## Содержание 

[Шаг 1. Загрузка данных](#section_id1)

[Шаг 2. Предобработка и исследовательский анализ](#section_id2)

[Шаг 3. Подготовка данных](#section_id3)

[Шаг 4. Обучение моделей](#section_id4)

[Шаг 5. Проверка на тестовой выборке](#section_id5)

[Шаг 6. Вывод](#section_id6)

In [1]:
# установка библиотек
!pip install -q spacy

In [2]:
# импорт библиотек

# работа с данными
import pandas as pd
import numpy as np

# подготовка данных
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

# модели машинного обучения
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# пайплайны
from sklearn.pipeline import Pipeline

# инструменты поиска
from sklearn.model_selection import GridSearchCV

# инструменты управления ресурсами
import joblib
import warnings

# метрика для оценки прогноза
from sklearn.metrics import f1_score

In [3]:
# константы
TEST_SIZE = 0.25 
RANDOM_STATE = 42

# настройки
warnings.filterwarnings('ignore')

<a id='section_id1'></a>
## Шаг 1. Загрузка данных
[к содержанию](#section_id)

In [4]:
# загрузка данных
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


<a id='section_id2'></a>
## Шаг 2. Предобработка и исследовательский анализ
[к содержанию](#section_id)

In [5]:
# удаление столбца без названия
df = df[['text', 'toxic']]

In [6]:
# функция для обзора данных
def preview(dataset):
    '''Функция принимает на вход набор данных и выводит основную информацию о нем.'''
    display(dataset.head())
    dataset.info()
    display(dataset.describe(include='all', datetime_is_numeric=True).T)

In [7]:
# обзор данных
preview(df)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
text,159292,159292,"Question about Will \n\nHi, do you think Will ...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
toxic,159292.0,NaN,NaN,NaN,0.101612,0.302139,0.0,0.0,0.0,0.0,1.0


In [8]:
# определение баланса классов
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Выводы о данных:
- пропусков нет
- повторяющихся комментариев нет
- типы данных приведены верно
- баланс классов смещен в сторону нетоксичных комментариев

<a id='section_id3'></a>
## Шаг 3. Подготовка данных
[к содержанию](#section_id)

In [9]:
# содание тренировочной и тестовой выборки
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['toxic'], axis=1),
    df['toxic'],
    test_size = TEST_SIZE, 
    random_state = RANDOM_STATE
)

In [10]:
# создание списков текстов
train_texts = X_train['text'].to_list()
test_texts = X_test['text'].to_list()

In [11]:
# загрузка инструментов для работы с английским языком
nlp = spacy.load("en_core_web_sm")

In [12]:
# функция для очистки и лемматизации текста
def lemm_text(texts):
    '''функция принимает на вход список текстов, 
    возвращает список ощищенных от символов, лемматизированных текстов.'''

    # очистка текста от всех символов, кроме латиницы
    clean_texts = [re.sub(r'[^a-zA-Z ]', ' ', text) for text in texts]
    
    # разбиение списка текстов на порции
    make_parts = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
    # разбиение на порции по 100 текстов
    text_parts = make_parts(clean_texts, 100)
    
    # слияние текстов с разделителем
    res = []
    for part in text_parts:
        united_texts = ' '.join([text + '*' for text in part])
        
        # лемматизация текстов после слияния
        nlp_text_object = nlp(united_texts)
        lemm_text_part = ' '.join([token.lemma_ for token in nlp_text_object])
        
        # наполнение списка результатов
        lemm_text_list = lemm_text_part.split('*')
        for text in lemm_text_list[:-1]:
            res.append(text)

    return res

In [13]:
%%time
# создание корпуса лемматизированной тренировочной выборки 
with joblib.parallel_backend("threading"):
    X_train_corpus = lemm_text(train_texts)

CPU times: user 23min 53s, sys: 15.2 s, total: 24min 8s
Wall time: 24min 10s


In [14]:
# проверка корректности лемматизации тренировочной выборки
print(len(X_train_corpus), len(train_texts))
print('=' * 50)
print(train_texts[0])
print('-' * 50)
print(X_train_corpus[0])
print('=' * 50)
print(train_texts[-1])
print('-' * 50)
print(X_train_corpus[-1])

119469 119469
The source was also not acceptable for reasons already stated. You also just rambled on about a load of nonsense.
--------------------------------------------------
the source be also not acceptable for reason already state   you also just ramble on about a load of nonsense 
"
I'm sorry, Tip; the deduction was obvious (and wrong). The article as written did not contain any links to reliable sources, depending instead on the subject's own website, amazon.com, and other notoriously unreliable sources. Remember also that Notability is not contagious; being the agent for a notable person does not make this guy notable. I'd suggest you rewrite this one in a sandbox, using more reliable sources; and if there are no such sources, consider what this says about the guy's notability. Your fellow cheesehead,   |  Talk "
--------------------------------------------------
    I m sorry   Tip   the deduction be obvious   and wrong    the article as write do not contain any link to reli

In [15]:
%%time
# создание корпуса лемматизированной тестовой выборки 
with joblib.parallel_backend("threading"):
    X_test_corpus = lemm_text(test_texts)

CPU times: user 8min 7s, sys: 6.69 s, total: 8min 14s
Wall time: 8min 14s


In [16]:
# проверка корректности лемматизации тестовой выборки
print(len(X_test_corpus), len(test_texts))
print('=' * 50)
print(test_texts[0])
print('-' * 50)
print(X_test_corpus[0])
print('=' * 50)
print(test_texts[3])
print('-' * 50)
print(X_test_corpus[3])

39823 39823
Sometime back, I just happened to log on to www.izoom.in with a friend’s reference and I was amazed to see the concept Fresh Ideas Entertainment has come up with. So many deals… all under one roof. This website is very user friendly and easy to use and is fun to be on.
You have Gossip, Games, Facts… Another exciting feature to add to it is Face of the Week… Every week, 4 new faces are selected and put up as izoom faces. It’s great to have been selected in four out of a group of millions. 
This new start up has already got many a deals in its kitty. Few of them being TheFortune Hotel, The Beach… are my personal favorites. izoom.in has a USP of mobile coupons. Coupons are available even when a user cannot access internet. You just need to SMS izoom support to 56767 and you get attended immediately.
All I can say is izoom.in is a must visit website for everyone before they go out for shopping or dining or for outing.
Cheers!!!
--------------------------------------------------

In [17]:
# инициализация экземпляра класса векторизатора
count = TfidfVectorizer(stop_words=set(stopwords.words('english')))

In [18]:
%%time
# преобразование корпуса слов тренировочной выборки
X_train_tf_idf = count.fit_transform(X_train_corpus)
# преобразование корпуса слов тестовой выборки
X_test_tf_idf = count.transform(X_test_corpus)

CPU times: user 7.31 s, sys: 40 ms, total: 7.35 s
Wall time: 7.36 s


<a id='section_id4'></a>
## Шаг 4. Обучение моделей
[к содержанию](#section_id)

In [19]:
# пайплайн обучения
pipe_final = Pipeline([
    ('models', LogisticRegression(random_state=RANDOM_STATE))
])

In [20]:
# задание параметров для пайплайна
param_grid = [
    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE,
            solver='liblinear',
            penalty='l2'
        )],
        'models__C': [5, 10, 15],
    },
    
    # словарь для модели DecisionTreeClassifier()
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_features': range(6, 8),
        'models__max_depth': range(8, 10)
    },
    
    # словарь для модели KNeighborsClassifier() 
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': [5, 25]   
    }
]

In [21]:
# инициализация подбора параметров
grid_search = GridSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring='f1',
    n_jobs=-1,
    verbose=1
)

In [22]:
%%time
# запуск подбора параметров
with joblib.parallel_backend("threading"):
        grid_search.fit(X_train_tf_idf, y_train)

print('Лучшая модель и её параметры:\n\n', grid_search.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', grid_search.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Лучшая модель и её параметры:

 Pipeline(steps=[('models',
                 LogisticRegression(C=15, random_state=42,
                                    solver='liblinear'))])
Метрика лучшей модели на тренировочной выборке: 0.766682041196146
CPU times: user 14min 25s, sys: 6min 30s, total: 20min 56s
Wall time: 20min 57s


In [23]:
# получение результатов лучших моделей
results = pd.DataFrame(grid_search.cv_results_)
results.sort_values(by='rank_test_score', inplace=True)
results.head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_models,param_models__C,param_models__max_depth,param_models__max_features,param_models__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
2,15.547536,0.601210,0.010678,0.000550,"LogisticRegression(C=15, random_state=42, solv...",15,NaN,NaN,NaN,"{'models': LogisticRegression(C=15, random_sta...",0.765125,0.759722,0.778465,0.760439,0.769659,0.766682,0.006893,1
1,15.649200,0.163290,0.011057,0.000410,"LogisticRegression(C=15, random_state=42, solv...",10,NaN,NaN,NaN,"{'models': LogisticRegression(C=15, random_sta...",0.764842,0.758653,0.778496,0.760000,0.767104,0.765819,0.007050,2
0,13.187956,0.767669,0.010389,0.000076,"LogisticRegression(C=15, random_state=42, solv...",5,NaN,NaN,NaN,"{'models': LogisticRegression(C=15, random_sta...",0.761120,0.752320,0.773723,0.756976,0.760610,0.760950,0.007121,3
7,0.032401,0.003908,101.710705,0.620134,KNeighborsClassifier(),NaN,NaN,NaN,5,"{'models': KNeighborsClassifier(), 'models__n_...",0.249917,0.250171,0.256110,0.251768,0.243852,0.250364,0.003940,4
8,0.033375,0.001149,101.527738,0.692606,KNeighborsClassifier(),NaN,NaN,NaN,25,"{'models': KNeighborsClassifier(), 'models__n_...",0.117375,0.116647,0.112273,0.103623,0.094716,0.108927,0.008628,5
3,0.115687,0.006541,0.030254,0.000666,DecisionTreeClassifier(random_state=42),NaN,8,6,NaN,{'models': DecisionTreeClassifier(random_state...,0.000000,0.001642,0.000821,0.000000,0.000000,0.000493,0.000657,6
5,0.116077,0.005737,0.029900,0.000246,DecisionTreeClassifier(random_state=42),NaN,9,6,NaN,{'models': DecisionTreeClassifier(random_state...,0.000000,0.001642,0.000821,0.000000,0.000000,0.000493,0.000657,6
4,0.116147,0.002403,0.032728,0.004552,DecisionTreeClassifier(random_state=42),NaN,8,7,NaN,{'models': DecisionTreeClassifier(random_state...,0.000000,0.000000,0.000821,0.000000,0.000000,0.000164,0.000329,8
6,0.132552,0.008930,0.031114,0.000797,DecisionTreeClassifier(random_state=42),NaN,9,7,NaN,{'models': DecisionTreeClassifier(random_state...,0.000000,0.000000,0.000821,0.000000,0.000000,0.000164,0.000329,8


<a id='section_id5'></a>
## Шаг 5. Проверка на тестовой выборке
[к содержанию](#section_id)

In [24]:
# проверка лучшей модели на тестовой выборке
pred = grid_search.best_estimator_.predict(X_test_tf_idf)

print("f1 тестовой выборки:", f1_score(y_test, pred))

f1 тестовой выборки: 0.7802182621909103


<a id='section_id6'></a>
## Шаг 6. Вывод
[к содержанию](#section_id)

Решена задача по классификации токсичных комментариев. При создании тренировочной и тестовой выборки выявлено, что токсичных коментариев значительно меньше, поэтому применена стратификация, чтобы равномерно распределить комментарии каждого класса по выборкам. Лемматизация произведена с помощью pymystem3 внутри функции, фильтровались специальные символы и стоп-слова. Для получения наборов признаков для обучения моделей использовался CountVectorizer, так как он показал лучшие результаты в сравнении с TfIdfVectorizer. Обучение производилось в пайплайне: исследовались модели LogisticRegression, DecisionTreeClassifier, KNeighborsClassifier. Лучшей моделью стала LogisticRegression с параметром регуляризации C=15. Неплохие результаты показала модель KNeighborsClassifier с параметрами n_neighbors=5. На тестовой выборке лучшая модель показала значение метрики f1_score равным 0.78

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны